# Exploration into D*-Lite in simulated 3D terrain
The purpose of this notebook is to explore the D* Lite algorithm and its performance as compared to A* when path-planning in 3D terrain. The initial path planning will be done on coarse estimated 3D terrain and updated as more fine detail is learned about the terrain.

### Autoreload modules
**ALWAYS RUN THIS DURING DEVELOPMENT.**

Run this cell to make sure that modules are automatically reloaded as changes are made and the kernel does not need to be restarted.

In [1]:
%load_ext autoreload
%autoreload 2

## Tasks
These are the following steps that have been and still need to be completed:
- [x] Creating random terrain 
  - [x] Terrain should have varying levels of resolution, scale, and detail
  - [x] Terrain should be generated at coarse and fine levels of detail
- [x] Creating a grid map
  - [x] Discretize each x,y,z point of map into nodes
  - [x] Map the successors, predecessors, and edges connected to each node
  - [x] Create a container which can house all nodes
- [x] Implementing A*
  - [x] Write A* algorithm using grid map and edge costs
  - [x] Assign start and goal and create path 
  - [x] Display A* path (nodes travelled) on terrain map
- [x] Implementing D*-Lite
  - [x] Updating edge costs to grid map
  - [x] Rescan ability to grid map
  - [x] Write D*-Lite algorithm using grid map and edge costs
  - [x] Assign start and goal and create path
  - [ ] Display D*-Lite path and recalculated path (nodes travelled) on terrain map
- [ ] Compare methods
  - [ ] Create quantititave performance analysis of computation time, cost of path
  - [ ] A* computed on fine terrain and performed on fine terrain (Baseline)
  - [ ] A* computed on coarse terrain and performed on fine terrain (Current approach)
  - [ ] A* computed on coarse terrain and performed on fine terrain with recalculating on update (Next approach) 
  - [ ] D*-Lite computed on coarse terrain and performed on fine terrain (Proposed approach)
- [ ] Possible simulation improvements
  - [ ] Add large obstacles to terrain
  - [ ] Adding enemy sightline simulation
  - [ ] Make simulated terrain non-square

## Creating Terrain
The first step is to create simulated terrain. A common technique to create random terrain is to use specific kinds of noises that create terrain-like shapes. In this example, we use simplex noise, one of the most commonly used typs of noise for generation of terrain.

### Import and Setup
Setup Jupyter Notebook with all important settings. 

To make graphic maps interactable, change
`%matplotlib inline` 
to
`%matplotlib qt`

In [2]:
%matplotlib qt
import noise
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from random import randrange

### Setup terrain hyperparameters
The noise consists of a different hyperparameters:
- resolution: How many "pixels" are used to represent terrain. *NOT* the coarseness or fineness of the map detail.
- scale: How large an area is being mapped (e.g `scale = 1.5` mean each "pixel"/node represents a 1.5x1.5 area).
- octaves: The level of detail of the terrain. Used to simulate coarse (small amount of octaves) and fine (large amount of octaves) detail of the terrain. An increase by 3 octaves from coarse to fine seems to result in the best looking terrain.
- random_seed: If random_seed is True, the offset is randomly chosen. If False, offset is set to 0. Python noise generation methods are not random and the same offset will produce the same results.
- min_height and max_height: The coarse map is scaled so the lowest point of the map is min_height and the highest point is max_height. The fine map is scaled to the coarse map to preserve the original relationship betwewn the values of the two maps.


In [3]:
resolution = 500
scale = 2.0
coarse_octaves = 1
fine_octaves = 4
random_seed = False
min_height = 0.0
max_height = 500.0

### Create terrain

The noise is generated according to the parameters as defined above.

In [4]:
if random_seed:
    offset = randrange(100)
else:
    offset = 0

shape = (resolution, resolution)
rough_world = np.zeros(shape)
fine_world = np.zeros(shape)

for i in range(resolution):
    for j in range(resolution):
        rough_world[i][j] = noise.snoise2(i*scale/resolution, 
                                    j*scale/resolution, 
                                    octaves=coarse_octaves,  
                                    base=offset)
        fine_world[i][j] = noise.snoise2(i*scale/resolution, 
                                    j*scale/resolution, 
                                    octaves=fine_octaves, 
                                    base=offset)

The noise is then scaled to min and max height to create terrain.

In [5]:
fine_world = (fine_world-np.min(rough_world)) / (np.max(rough_world)-np.min(rough_world)) #scale fine world according to coarse world
fine_world = min_height + (max_height-min_height) * fine_world

rough_world = (rough_world-np.min(rough_world)) / (np.max(rough_world)-np.min(rough_world)) #scale coarse world between 0.0 and 1.0
rough_world = min_height + (max_height-min_height) * rough_world

### Display Terrain
We now display the terrain both as 2D bird'eye views of the terrain and 3D views of the terrain.
#### Bird's Eye View

In [6]:
fig_t_be, axs = plt.subplots(1,2)
fig_t_be.suptitle("Bird's eye view of generated terrain")
axs[0].imshow(rough_world,cmap='terrain')
axs[0].set_title('Coarse Terrain Estimate')             
axs[1].imshow(fine_world,cmap='terrain')
axs[1].set_title('Fine Terrain Detail')    

Text(0.5, 1.0, 'Fine Terrain Detail')

#### 3D View of Terrain

In [7]:
lin_x = np.linspace(0,resolution,resolution,endpoint=False)
lin_y = np.linspace(0,resolution,resolution,endpoint=False)
x,y = np.meshgrid(lin_x,lin_y)

fig_t_3d = plt.figure()
fig_t_3d.suptitle("3D View of Terrain")
ax_rough = fig_t_3d.add_subplot(121, projection="3d")
ax_rough.plot_surface(y,x,rough_world,cmap='terrain')
ax_rough.set_title('Coarse Terrain Estimate')

ax_fine = fig_t_3d.add_subplot(122, projection="3d")
ax_fine.plot_surface(y,x,fine_world,cmap='terrain')
ax_fine.set_title('Fine Terrain Detail')

Text(0.5, 0.92, 'Fine Terrain Detail')

## Creating Grid Map
This section is implemented in `src/utils.py` and `src/grid.py`

In [8]:
from src.grid import Grid

rough_grid = Grid(rough_world)
fine_grid = Grid(fine_world)

## A* Path Planning
This section is implemented in `src/astar.py`.

In [9]:
from src.astar import AStar

curr_map = fine_world

pathplanner = AStar(curr_map)

In [ ]:
path_baseline = pathplanner.get_path((10,10), (resolution-10,resolution-10))
arr_baseline = pathplanner.get_plt_arrays()

xx_baseline = arr_baseline[0, :]
yy_baseline = arr_baseline[1, :]
zz_baseline = arr_baseline[2, :]

In [11]:
fig_as = plt.figure()
fig_as.suptitle("Terrain with Baseline A* Path")

ax = fig_as.add_subplot(121)
ax.imshow(curr_map,cmap='terrain')
ax.plot(yy_baseline,xx_baseline, 'r')
ax.plot(yy_baseline[0],xx_baseline[0],'r*')
ax.plot(yy_baseline[-1],xx_baseline[-1],'rX')
ax.set_title("Bird's Eyeview Terrain with Path")

ax3 = fig_as.add_subplot(122, projection='3d')
ax3.plot_surface(y,x,curr_map,cmap='terrain')
ax3.plot(xx_baseline,yy_baseline,zz_baseline,'r.')
ax3.plot(xx_baseline[0],yy_baseline[0],zz_baseline[0],'r*')
ax3.plot(xx_baseline[-1],yy_baseline[-1],zz_baseline[-1],'rX')
ax3.set_title('Terrain with Path')

Text(0.5, 0.92, 'Terrain with Path')

## D*-Lite Pathplanning
This section is implemented in `src/dsl.py`

In [12]:
from src.dsl import DStarLite

known_map = rough_world
true_map = fine_world

dsl_planner = DStarLite(known_map, true_map)

In [ ]:
path_dsl = dsl_planner.get_path((10,10), (resolution-10,resolution-10))
arr_dsl = dsl_planner.get_plt_arrays()

xx_dsl = arr_dsl[0, :]
yy_dsl = arr_dsl[1, :]
zz_dsl = arr_dsl[2, :]

In [15]:
fig_dsl = plt.figure()
fig_dsl.suptitle("Terrain with D*-Lite Path")

ax = fig_dsl.add_subplot(121)
ax.imshow(true_map,cmap='terrain')
ax.plot(yy_dsl,xx_dsl, 'r')
ax.plot(yy_dsl[0],xx_dsl[0],'r*')
ax.plot(yy_dsl[-1],xx_dsl[-1],'rX')
ax.set_title("Bird's Eyeview Terrain with Path")

ax3 = fig_dsl.add_subplot(122, projection='3d')
ax3.plot_surface(y,x,true_map,cmap='terrain')
ax3.plot(xx_dsl,yy_dsl,zz_dsl,'r.')
ax3.plot(xx_dsl[0],yy_dsl[0],zz_dsl[0],'r*')
ax3.plot(xx_dsl[-1],yy_dsl[-1],zz_dsl[-1],'rX')
ax3.set_title('Terrain with Path')

Text(0.5, 0.92, 'Terrain with Path')

In [16]:
final_known_terrain = dsl_planner.known_grid.to_np()

fig_learn = plt.figure()
fig_learn.suptitle("Map of what DSL knows")
ax1 = fig_learn.add_subplot(131)
ax1.imshow(known_map, cmap='terrain')
ax2 = fig_learn.add_subplot(132)
ax2.imshow(final_known_terrain, cmap='terrain')
ax3 = fig_learn.add_subplot(133)
ax3.imshow(true_map, cmap='terrain')